<a href="https://colab.research.google.com/github/wallybeamm/neural_entity_recognition/blob/feature%2Finit/enitity_recognation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
print('Hello World')

Hello World


In [14]:
# Perform standard imports 
import spacy 
nlp = spacy.load('en_core_web_sm')
# Write a function to display basic entity info: 
def show_ents(doc): 
  if doc.ents: 
    for ent in doc.ents: print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
  else: 
    print('No named entities found.')
doc1 = nlp("Apple is looking at buying U.K. startup for $1 billion") 
show_ents(doc1)


Apple - 0 - 5 - ORG - Companies, agencies, institutions, etc.
U.K. - 27 - 31 - GPE - Countries, cities, states
$1 billion - 44 - 54 - MONEY - Monetary values, including unit


In [15]:
TRAIN_DATA = [
    ('Who is Nishanth?', {
        'entities': [(7, 15, 'PERSON')]
    }),
     ('Who is Kamal Khumar?', {
        'entities': [(7, 19, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]

In [16]:
model = None
output_dir = "/content/Model"
n_iter = 100

In [17]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')


Created blank 'en' model


In [18]:
import random
from spacy.training.example import Example

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
            for text, annotations in batch:
                # create Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=0.3)
                
        print(losses)

{'ner': 12.867681920528412}
{'ner': 12.269958734512329}
{'ner': 10.988915383815765}
{'ner': 9.58344829082489}
{'ner': 7.824333682656288}
{'ner': 5.664001441560686}
{'ner': 4.534312762320042}
{'ner': 3.968154416041216}
{'ner': 3.4042938351758494}
{'ner': 6.786874026679087}
{'ner': 4.979112587694544}
{'ner': 2.6489539966637494}
{'ner': 2.3734214174480712}
{'ner': 1.8121026793142647}
{'ner': 2.496927369498411}
{'ner': 1.8466328102162137}
{'ner': 1.2956255437302175}
{'ner': 0.9753100419482656}
{'ner': 0.8017581532767508}
{'ner': 0.6856597603391705}
{'ner': 1.293757607830191}
{'ner': 0.5172842797017543}
{'ner': 0.23686811432660237}
{'ner': 0.07120647410648115}
{'ner': 0.07949336052351132}
{'ner': 0.025343809040388635}
{'ner': 0.0023954455297810102}
{'ner': 0.0017413423311672485}
{'ner': 2.7409106156041146e-05}
{'ner': 0.0002592851821628674}
{'ner': 0.020918435156208483}
{'ner': 2.98372796649386e-05}
{'ner': 2.5923227657486798e-05}
{'ner': 0.00011024153521565163}
{'ner': 2.9028798676471964e-

In [19]:
# TEST
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Nishanth', 'PERSON')]
Entities [('London', 'LOC'), ('Berlin', 'LOC')]
Entities [('Kamal Khumar', 'PERSON')]


In [22]:
if output_dir is not None:
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to /content/Model
